TODO:
- [ ] framework
    - [ ] documentation
    - Kinetics
        - [x] reform struct with number of gates
    - [x] simple channel -> ih, il, ina, iltk
    - [x] copmlex channel -> ihtk, iltk
    - [x] generic channel -> ik
    - [x] misc
        - [x] `function dof(channels::Vector{AbstractIonChannel})`
        - [x] `function simpleConductanceModel(channels::Vector{AbstractIonChannel}, stim::Function)`
- [ ] channel zoo
    - [x] ihtk
    - [x] ih
    - [x] ik
    - [x] il
    - [x] iltk
        - [x] regular
        - [ ] complex
    - [x] ina `hh_sodium`

In [1]:
using DifferentialEquations
using Plots
gr(fmt="png")

include("../src/HHModel.jl")

Main.HHModel

In [2]:
# ltk = HHModel.low_voltage_gated_potassium(0);
# ltk_cmplx = HHModel.low_voltage_gated_potassium(10, iltkcomplex=true);
htk = HHModel.high_voltage_gated_potassium(2.8);
ina = HHModel.hh_sodium(13.0);
# ik = HHModel.hh_potassium(0);
ih = HHModel.ihcurrent(0.43);
il = HHModel.leakage(0.03);

stimulus = (t, param) -> begin
    if (param.start < t) & (t < (param.start + param.dur))
        return param.amp
    else
        return 0
    end
end

_model = HHModel.simpleConductanceModel([htk, ina, ih, il], stimulus)

#45 (generic function with 1 method)

---

In [5]:
_p = (E=(sodium=81.27, potassium=-80.78, ih=-26, leak=-65), 
      stim=(start=50, amp=0, dur=100),)

tspan = (0.0, 200.0)
v0 = -60.0
u0 = HHModel.setup_init([htk, ina, ih, il], v0)

prob = ODEProblem(_model, u0, tspan, _p)
@time sol = solve(prob, Tsit5(), dt=0.1);


InterruptException: InterruptException:

In [ ]:
plot(sol, vars=(1), legend=nothing)

In [ ]:
not = (x) -> !x
function simpleConductanceModel(channels::Vector{T}, stim::Function) where {T <: HHModel.AbstractIonChannel}
    nchannel = length(channels)
    nvar = HHModel.dof(channels)
    
    return (du, u, p, t) -> begin
        v = u[1]
        param = u[2:end-1]
        var_idx = 1
        
        _current = zeros(nchannel)
        for (idx, item) in enumerate(channels)
            _var_step = sum(HHModel.dof(item))
            println(item.name, " ", _var_step)
            println(length(param[var_idx:var_idx-1+_var_step]))
            println(var_idx)
            (_icurrent, _iderivitate) = HHModel.step(item, V=v, var=param[var_idx:var_idx-1+_var_step], E=p.E[item.ion])
            _current[idx] = _icurrent
            du[var_idx:var_idx-1+_var_step] = _iderivitate[not.(isnothing.(_iderivitate))]
            var_idx += _var_step
        end
        
        I = stim(t, p.stim)
        du[1] = I - sum(_current)
        u[end] = I
        
        du, u, p, t
    end
end

---

### generic channel
ik

In [ ]:
ik_rule = (knt, var) -> begin
    (a,b,c) = var
    a ^ 4 * b * c
end

_a_tau = (V) -> 100*(7*exp((V+60)/14)+29*exp(-(V+60)/24))^(-1)+0.1
_a_infty = (V) -> (1+exp(-(V+31)/6))^(-1/4)
_a = HHModel.Kinetics(4, _a_infty, _a_tau)

_b_tau = (V) -> 1000*(14*exp((V+60)/27)+29*exp(-(V+60)/24))^(-1)+1;
_b_infty = (V) -> (1+exp((V+66)/7))^(-1/2);
_b = HHModel.Kinetics(1, _b_infty, _b_tau)

_c_tau = (V) -> 90*(1+exp(-(V+66)/17))^(-1)+10;
_c_infty = (V) -> (1+exp((V+66)/7))^(-1/2);
_c = HHModel.Kinetics(1, _c_infty, _c_tau)

ik = HHModel.GenericIonChannel("hh potassium", :potassium,
    10, ik_rule,
    [_a, _b, _c]
);

In [ ]:
HHModel.step(ik, V=-10, var=[0.1, 0.2, 0.1], E=-80)

### complex channel
_ihtk


In [ ]:
_n_tau = (V) -> 100*(11*exp((V+60)/24)+21*exp(-(V+60)/23))^(-1)+0.7;
_n_infty = (V) -> (1+exp(-(V+15)/5))^(-1/2)
_n = HHModel.Kinetics(2, _n_infty, _n_tau)

_p_tau = (V) -> 100*(4*exp((V+60)/32)+5*exp(-(V+60)/22))^(-1)+5;
_p = HHModel.Kinetics(1, -23, 6, _tau=_p_tau)

ihtk = HHModel.ComplexIonChannel("high voltage gated potassium", :potassium,
       10.0, [0.5, 0.5],
       [(_n, HHModel.Kinetics()), 
        (_p, HHModel.Kinetics())]
);

In [ ]:
HHModel.step(ihtk, V=-1, var=[0.1, 0.2], E=-80)

In [ ]:
plot(ihtk.var[1][1].tau, -100, 20)

### simple channel

In [ ]:
_m_tau = (V) -> 10 / (5*exp((V+60)/18)+36*exp(-(V+60)/25))+0.04
_m = HHModel.Kinetics(3, -38.0, 7.0, _tau = _m_tau)
    
_h_tau = (V) -> 100 / (7*exp((V+60)/11)+10*exp(-(V+60)/25))+0.6
_h = HHModel.Kinetics(1, -65.0, 6.0, _tau=_h_tau, state=:inactivation)
    
hh_sodium = HHModel.SimpleIonChannel("hh sodium", :sodium, 
        10, _m, _h);

In [ ]:
HHModel.dof(hh_sodium)

In [ ]:
HHModel.step(hh_sodium, V=10, var=[0.1, 0.4], E=28)

In [ ]:
HHModel.dof([hh_sodium, ik, ihtk])

---

In [ ]:
ih = HHModel.ihcurrent(10);

In [ ]:
plot(ih.h.infty, -150, 20)

In [ ]:
ltk = HHModel.low_voltage_gated_potassium(15.0, subtype=:kv1);

# plot((x) -> HHModel.time_constant(ltk.m, x), -150, 20, legend=nothing)    
plot(ltk.m.infty, -150, 20, legend=nothing, ylim=(0, 1))

In [ ]:
hhna = HHModel.hh_sodium(20);

# plot(hhna.h.infty, -150, 20)
# plot!(hhna.m.infty, -150, 20)
plot((x) -> hhna.m.infty(x), -150, 20)